## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thanh Hoa,VN,19.8000,105.7667,69.78,71,100,6.02,overcast clouds
1,1,Bluff,NZ,-46.6000,168.3333,55.80,90,97,13.85,overcast clouds
2,2,Albany,US,42.6001,-73.9662,71.64,91,67,4.90,broken clouds
3,3,Barrow,US,71.2906,-156.7887,15.82,79,100,16.11,light snow
4,4,Hermanus,ZA,-34.4187,19.2345,76.53,36,3,2.95,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hermanus,ZA,-34.4187,19.2345,76.53,36,3,2.95,clear sky
6,6,Bambous Virieux,MU,-20.3428,57.7575,77.25,73,75,12.66,broken clouds
10,10,Maunabo,PR,18.0072,-65.8993,84.47,80,26,13.20,scattered clouds
11,11,Phan Thiet,VN,10.9333,108.1000,81.28,85,100,3.65,moderate rain
12,12,Rikitea,PF,-23.1203,-134.9692,77.23,70,56,21.54,broken clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Camabatela,AO,-8.1881,15.3750,80.33,34,100,4.76,overcast clouds
673,673,Siocon,PH,7.7061,122.1353,79.61,87,93,2.91,overcast clouds
674,674,Turan,VN,16.0678,108.2208,78.78,69,40,5.75,scattered clouds
676,676,Canutama,BR,-6.5339,-64.3831,76.28,97,100,2.89,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                233
City                   233
Country                233
Lat                    233
Lng                    233
Max Temp               233
Humidity               233
Cloudiness             233
Wind Speed             233
Current Description    233
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hermanus,ZA,76.53,clear sky,-34.4187,19.2345,
6,Bambous Virieux,MU,77.25,broken clouds,-20.3428,57.7575,
10,Maunabo,PR,84.47,scattered clouds,18.0072,-65.8993,
11,Phan Thiet,VN,81.28,moderate rain,10.9333,108.1000,
12,Rikitea,PF,77.23,broken clouds,-23.1203,-134.9692,
16,Port-Gentil,GA,85.96,broken clouds,-0.7193,8.7815,
17,Delmas,HT,83.12,scattered clouds,18.5447,-72.3028,
23,Nicoya,CR,82.31,moderate rain,10.1483,-85.4520,
24,East London,ZA,76.14,clear sky,-33.0153,27.9116,
26,Iracoubo,GF,82.62,overcast clouds,5.4802,-53.2011,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")     

Hotel not found... skipping.60
Hotel not found... skipping.66
Hotel not found... skipping.156
Hotel not found... skipping.170
Hotel not found... skipping.208
Hotel not found... skipping.223
Hotel not found... skipping.273
Hotel not found... skipping.305
Hotel not found... skipping.311
Hotel not found... skipping.364
Hotel not found... skipping.399
Hotel not found... skipping.435
Hotel not found... skipping.471


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hermanus,ZA,76.53,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
6,Bambous Virieux,MU,77.25,broken clouds,-20.3428,57.7575,Casa Tia Villa
10,Maunabo,PR,84.47,scattered clouds,18.0072,-65.8993,Maunacaribe
11,Phan Thiet,VN,81.28,moderate rain,10.9333,108.1000,Hotel Saigon-PT
12,Rikitea,PF,77.23,broken clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
671,Camabatela,AO,80.33,overcast clouds,-8.1881,15.3750,Comércio e Prestação de Serviços
673,Siocon,PH,79.61,overcast clouds,7.7061,122.1353,ABS Hotel
674,Turan,VN,78.78,scattered clouds,16.0678,108.2208,Grand Mercure Danang
676,Canutama,BR,76.28,heavy intensity rain,-6.5339,-64.3831,Hotel Jhennifer


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))